Referensi
1. https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/
2. https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [583]:
# Mengimpor library & package yang digunakan

import re
import nltk
import math
import string
import stanza
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint
from wordcloud import WordCloud
from IPython.display import display
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## **A - Edit Distance (40 Poin)**

### 1. [10] Buatlah sebuah fungsi “edit_distance”. Fungsi tersebut memiliki dua buah parameter string yaitu string_1 dan string_2. Fungsi ini dapat menghitung Levenshtein Distance antara string_1 dengan string_2. Lebih jelasnya lagi, fungsi tersebut akan mengembalikan nilai terkecil yang dibutuhkan untuk mentransformasi string_1 menjadi string_2.

In [584]:
def edit_distance(string_1, string_2):
    """
    Reference: https://www.datacamp.com/community/tutorials/fuzzy-string-python
    """

    # Initialize matrix of zeros
    rows = len(string_1)+1
    cols = len(string_2)+1
    distances = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distances[i][0] = i
            distances[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions
    for col in range(1, cols):
        for row in range(1, rows):
            if string_1[row-1] == string_2[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                cost = 1

            distances[row][col] = min(
                distances[row-1][col] + 1,       # Cost of deletions
                distances[row][col-1] + 1,       # Cost of insertions
                distances[row-1][col-1] + cost   # Cost of substitutions
            )

    pprint(distances)

    return distances[row][col]


### 2. [5] Menggunakan fungsi yang telah dibuat pada soal sebelumnya, carilah nilai edit_distance dari pasangan kata berikut ini:

### a. phasmophobia - puafnilhotik

In [585]:
edit_distance("phasmophobia", "puafnilhotik")

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 2,  1,  1,  2,  3,  4,  5,  6,  6,  7,  8,  9, 10],
       [ 3,  2,  2,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 4,  3,  3,  2,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 5,  4,  4,  3,  3,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 6,  5,  5,  4,  4,  4,  4,  5,  6,  6,  7,  8,  9],
       [ 7,  6,  6,  5,  5,  5,  5,  5,  6,  7,  7,  8,  9],
       [ 8,  7,  7,  6,  6,  6,  6,  6,  5,  6,  7,  8,  9],
       [ 9,  8,  8,  7,  7,  7,  7,  7,  6,  5,  6,  7,  8],
       [10,  9,  9,  8,  8,  8,  8,  8,  7,  6,  6,  7,  8],
       [11, 10, 10,  9,  9,  9,  8,  9,  8,  7,  7,  6,  7],
       [12, 11, 11, 10, 10, 10,  9,  9,  9,  8,  8,  7,  7]])


7

### b. genetik - ganteng

In [586]:
edit_distance("genetik", "ganteng")

array([[0, 1, 2, 3, 4, 5, 6, 7],
       [1, 0, 1, 2, 3, 4, 5, 6],
       [2, 1, 1, 2, 3, 3, 4, 5],
       [3, 2, 2, 1, 2, 3, 3, 4],
       [4, 3, 3, 2, 2, 2, 3, 4],
       [5, 4, 4, 3, 2, 3, 3, 4],
       [6, 5, 5, 4, 3, 3, 4, 4],
       [7, 6, 6, 5, 4, 4, 4, 5]])


5

### 3. [13] Salah satu metode untuk menghitung edit distance secara heuristik adalah menggunakan “keyboard distance”. Penentuan “bobot” atau “weight” pada metode ini dapat bervariasi. Penggunaan graf dari keyboard (asumsikan QWERTY keyboard) merupakan salah satu metode yang dapat dilakukan.

### Pada graf yang akan digunakan, satu node merepresentasikan satu key. Kemudian, satu edge menandakan bahwa dua key berdekatan yang artinya memiliki bobot edit distance sebesar 1.

### Buatlah matriks edit distance M dari suatu QWERTY keyboard! M(ii, jj) menyatakan jarak terpendek antara huruf “ii” ke huruf “jj” dalam graf QWERTY keyboard.

In [587]:
qwerty_keyboard = [
    ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p'],
    ['a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l'],
    [' ', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ' '],
]

alphabet = list(string.ascii_lowercase)


def getCharacterCoord(char, array):
    ii, jj = -1, -1
    for row in array:
        if char in row:
            ii = array.index(row)
            jj = row.index(char)
            return (ii, jj)
    raise ValueError(char + " not found in given keyboard layout")

def euclideanKeyboardDistance(c1, c2):
    coord1 = getCharacterCoord(c1, qwerty_keyboard)
    coord2 = getCharacterCoord(c2, qwerty_keyboard)
    return round(((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)**(0.5))

def M(ii, jj):
    qwerty_keyboard = [
        []
    ]

edit_distance_sum = 0

w1 = 'phasmophobia'
w2 = 'puafnilhotik'
# w1 = 'genetik'
# w2 = 'ganteng'

for i in range(len(w1)):
    distance = euclideanKeyboardDistance(w1[i], w2[i])
    # print(distance)
    if distance:
        edit_distance_sum += distance

print(edit_distance_sum)
print(euclideanKeyboardDistance('p', 'z'))

15
8


### 4. [5] Menggunakan matriks M yang telah dibuat pada soal sebelumnya, carilah nilai edit_distance dari pasangan kata berikut ini:

### a. phasmophobia - puafnilhotik

### b. genetik - ganteng

### 5. [7] Bandingkan hasil yang kamu temui pada nomor A2 dan A4. Apa yang dapat kamu simpulkan terkait kedua metode edit distance ini? Selain itu, apakah kedua algoritma edit distance ini sudah sempurna? Jelaskan analisis singkat kamu minimal dalam 3 kalimat.

## **B - N-Gram Language Model (30 Poin)**

### Perhatikan empat kalimat berikut
### K1 : orang itu menjadi kepala rumah sakit di desa
### K2 : kepala orang itu sedang sakit di rumah sakit
### K3 : kepala rumah sakit itu memakan kelapa bersama warga desa
### K4 : orang sakit itu memakan kelapa bersama kepala rumah sakit

### 1. [12] Jika diberikan threshold sebesar 10^-12 dengan menggunakan language model bigram dan trigram, manakah di antara kalimat berikut ini yang memiliki kecenderungan pada spelling error? Tunjukkan peluang dari setiap kalimat berikut dan tentukan kalimat yang mengandung spelling error!

In [588]:
sentences = [
    'orang itu menjadi kepala rumah sakit di desa',
    'kepala orang itu sedang sakit di rumah sakit',
    'kepala rumah sakit itu memakan kelapa bersama warga desa',
    'orang sakit itu memakan kelapa bersama kepala rumah sakit',
]

unigrams = [
    row.split(' ') for row in sentences
]

bigrams = map(
    lambda row: list(nltk.ngrams(row.split(' '), 2)),
    sentences
)

trigrams = map(
    lambda row: list(nltk.ngrams(row.split(' '), 3)),
    sentences
)

unigrams_sr = pd.Series(unigrams)
bigrams_sr = pd.Series(bigrams)
trigrams_sr = pd.Series(trigrams)

print(list(unigrams_sr), "\n")
print(list(bigrams_sr), "\n")
print(list(trigrams_sr), "\n")

[['orang', 'itu', 'menjadi', 'kepala', 'rumah', 'sakit', 'di', 'desa'], ['kepala', 'orang', 'itu', 'sedang', 'sakit', 'di', 'rumah', 'sakit'], ['kepala', 'rumah', 'sakit', 'itu', 'memakan', 'kelapa', 'bersama', 'warga', 'desa'], ['orang', 'sakit', 'itu', 'memakan', 'kelapa', 'bersama', 'kepala', 'rumah', 'sakit']] 

[[('orang', 'itu'), ('itu', 'menjadi'), ('menjadi', 'kepala'), ('kepala', 'rumah'), ('rumah', 'sakit'), ('sakit', 'di'), ('di', 'desa')], [('kepala', 'orang'), ('orang', 'itu'), ('itu', 'sedang'), ('sedang', 'sakit'), ('sakit', 'di'), ('di', 'rumah'), ('rumah', 'sakit')], [('kepala', 'rumah'), ('rumah', 'sakit'), ('sakit', 'itu'), ('itu', 'memakan'), ('memakan', 'kelapa'), ('kelapa', 'bersama'), ('bersama', 'warga'), ('warga', 'desa')], [('orang', 'sakit'), ('sakit', 'itu'), ('itu', 'memakan'), ('memakan', 'kelapa'), ('kelapa', 'bersama'), ('bersama', 'kepala'), ('kepala', 'rumah'), ('rumah', 'sakit')]] 

[[('orang', 'itu', 'menjadi'), ('itu', 'menjadi', 'kepala'), ('menjad

In [589]:
unigrams_sr_explode = unigrams_sr.explode()
unigrams_sr_value_counts = unigrams_sr_explode.value_counts()

bigrams_sr_explode = bigrams_sr.explode()
bigrams_sr_value_counts = bigrams_sr_explode.value_counts()

trigrams_sr_explode = trigrams_sr.explode()
trigrams_sr_value_counts = trigrams_sr_explode.value_counts()

# print(unigrams_sr_explode, "\n")
# print(bigrams_sr_explode, "\n")
# print(trigrams_sr_explode, "\n")
print(unigrams_sr_value_counts, "\n")
print(bigrams_sr_value_counts, "\n")
print(trigrams_sr_value_counts, "\n")

sakit      6
itu        4
kepala     4
rumah      4
orang      3
di         2
desa       2
memakan    2
kelapa     2
bersama    2
menjadi    1
sedang     1
warga      1
dtype: int64 

(rumah, sakit)       4
(kepala, rumah)      3
(orang, itu)         2
(sakit, itu)         2
(sakit, di)          2
(kelapa, bersama)    2
(memakan, kelapa)    2
(itu, memakan)       2
(orang, sakit)       1
(warga, desa)        1
(bersama, warga)     1
(sedang, sakit)      1
(di, rumah)          1
(itu, menjadi)       1
(itu, sedang)        1
(kepala, orang)      1
(di, desa)           1
(menjadi, kepala)    1
(bersama, kepala)    1
dtype: int64 

(kepala, rumah, sakit)        3
(memakan, kelapa, bersama)    2
(itu, memakan, kelapa)        2
(sakit, itu, memakan)         2
(orang, itu, menjadi)         1
(di, rumah, sakit)            1
(kelapa, bersama, kepala)     1
(orang, sakit, itu)           1
(bersama, warga, desa)        1
(kelapa, bersama, warga)      1
(rumah, sakit, itu)           1
(sakit, di, 

In [590]:
def count_occurence(tup_src, model):
    """
    Count occurences of a n-gram in the model provided

    params
    tup_src: tuple n-gram yang ingin dicek
    model: model yang menjadi referensi
    """
    try:
        return model[tup_src]
    except KeyError:
        return 0

def count_bigrams_prob(bigrams):
    """
    Count bigrams probability relative to the model di atas

    params
    bigrams: bigram yang pengen diitung prob-nya
    """
    bigrams_prob = (
        count_occurence(bigrams_a[0][0], unigrams_sr_value_counts)/
        unigrams_sr_value_counts.size
    )

    for bigram in bigrams:
        count_bigram = count_occurence(bigram, bigrams_sr_value_counts)
        count_word = count_occurence(bigram[0], unigrams_sr_value_counts)
        count_prob = (count_bigram + 1)/(count_word + len(bigram[0]))

        if count_prob:
            bigrams_prob *= count_prob

    return bigrams_prob

def count_trigrams_prob(trigrams):
    """
    Count trigrams probability relative to the model di atas

    params
    trigrams: trigram yang pengen diitung prob-nya
    """
    trigrams_prob = (
        count_occurence(trigrams_a[0][0], unigrams_sr_value_counts)/
        unigrams_sr_value_counts.size
    )

    for trigram in trigrams:
        count_trigram = count_occurence(trigram, trigrams_sr_value_counts)
        count_word = count_occurence(trigram[0], unigrams_sr_value_counts)
        count_prob = (count_trigram + 1)/(count_word + len(trigram[0]))

        if count_prob:
            trigrams_prob *= count_prob

    return trigrams_prob


### a. warga desa sedang sakit kelapa di rumah sakit

In [591]:
sentence_a = 'warga desa sedang sakit kelapa di rumah sakit'

bigrams_a = list(nltk.ngrams(sentence_a.split(' '), 2))
trigrams_a = list(nltk.ngrams(sentence_a.split(' '), 3))

bigrams_prob = count_bigrams_prob(bigrams_a)
trigrams_prob = count_trigrams_prob(bigrams_a)

print(bigrams_prob)
print(trigrams_prob)

3.854170520837187e-06
9.635426302092967e-08


b. orang itu sedang berada di rumah sakit

In [592]:
sentence_b = 'orang itu sedang berada di rumah sakit'

bigrams_b = list(nltk.ngrams(sentence_b.split(' '), 2))
trigrams_b = list(nltk.ngrams(sentence_b.split(' '), 3))

bigrams_prob = count_bigrams_prob(bigrams_b)
trigrams_prob = count_trigrams_prob(bigrams_b)

print(bigrams_prob)
print(trigrams_prob)

5.450898308041165e-05
9.084830513401942e-07


c. rumah sakit di desa itu tempat warga memakan kepala

In [593]:
sentence_c = 'rumah sakit di desa itu tempat warga memakan kepala'

bigrams_c = list(nltk.ngrams(sentence_c.split(' '), 2))
trigrams_c = list(nltk.ngrams(sentence_c.split(' '), 3))

bigrams_prob = count_bigrams_prob(bigrams_c)
trigrams_prob = count_trigrams_prob(bigrams_c)

print(bigrams_prob)
print(trigrams_prob)

4.282411689819096e-07
1.4274705632730318e-08


d. orang sakit sedang memakan kelapa bersama warga desa

In [594]:
sentence_d = 'orang sakit sedang memakan kelapa bersama warga desa'

bigrams_d = list(nltk.ngrams(sentence_d.split(' '), 2))
trigrams_d = list(nltk.ngrams(sentence_d.split(' '), 3))

bigrams_prob = count_bigrams_prob(bigrams_d)
trigrams_prob = count_trigrams_prob(bigrams_d)

print(bigrams_prob)
print(trigrams_prob)

2.3125023125023124e-06
3.2118087673643226e-08


### 2. [8] Dalam kasus umum (tidak terbatas pada soal ini), menurut kamu, manakah yang lebih baik; language model unigram, bigram, atau trigram? Berikan alasan pendukung jawaban kamu minimal dalam 3 kalimat.

### 3. [10] Menurut kamu apakah N-gram language model yang standar sudah baik untuk diterapkan pada bahasa yang kaya secara morfologis, seperti Bahasa Indonesia? Apa saja masalah berkaitan dengan morfologis yang mungkin ditemukan pada kasus ini? Ceritakan strategi kamu dalam mengatasinya dikaitkan dengan pengetahuan yang telah didapatkan pada bagian Text Processing minimal dalam 3 kalimat.

## **C - Pemetaan Dokumen dan Teks (30 Poin)**

### Dokumen A
<blockquote>
Dokumen A Sivitas Akademika Universitas Indonesia menjalankan aktivitas perkuliahan secara daring yang diikuti oleh mahasiswa dari seluruh jenjang. Hal ini diakibatkan karena adanya pandemi COVID-19 di Indonesia selama satu setengah tahun belakang.
</blockquote>

### Dokumen B
<blockquote>
Masyarakat memiliki respon berbeda dalam menyikapi pandemi COVID-19. Salah satu mahasiswa bernama Dipsy mengatakan kesulitan untuk belajar dengan maksimal karena sulit untuk berdiskusi dengan orang yang lebih pandai.
</blockquote>

### Dokumen C
<blockquote>
Salah satu kunci untuk menghadapi musibah adalah dengan bersabar dan yakin semua musibah akan berlalu pada waktunya. Badai pasti berlalu, jadi harus tetap semangat.
</blockquote>

### 1. [10] Buatlah pemetaan seluruh kata yang sudah dalam bentuk kata dasar lowercase dan bukan termasuk dalam stopwords dengan jumlah kemunculan kata tersebut di setiap dokumen. Kamu dibebaskan bagaimana cara menampilkannya, (tidak harus dalam bentuk tabel, bisa saja dalam bentuk key-value pair).

In [595]:
documents = [
    'Dokumen A Sivitas Akademika Universitas Indonesia menjalankan aktivitas perkuliahan secara daring yang diikuti oleh mahasiswa dari seluruh jenjang. Hal ini diakibatkan karena adanya pandemi COVID-19 di Indonesia selama satu setengah tahun belakang.',
    'Masyarakat memiliki respon berbeda dalam menyikapi pandemi COVID-19. Salah satu mahasiswa bernama Dipsy mengatakan kesulitan untuk belajar dengan maksimal karena sulit untuk berdiskusi dengan orang yang lebih pandai.',
    'Salah satu kunci untuk menghadapi musibah adalah dengan bersabar dan yakin semua musibah akan berlalu pada waktunya. Badai pasti berlalu, jadi harus tetap semangat.',
]

documents_sr = pd.Series(documents)

In [596]:
# Perform uncasing
cleaned_documents = documents_sr.str.lower()

In [597]:
# Tokenize documents
cleaned_documents_tokenized = cleaned_documents.map(
    lambda document: nltk.word_tokenize(document)
)

In [598]:
# Remove stopwords
stopwords = pd.read_csv('stopwords.txt', header=None)

cleaned_documents_tokenized_no_sw = cleaned_documents_tokenized.copy(deep=True)

for index, document in cleaned_documents_tokenized_no_sw.iteritems():
    cleaned_documents_tokenized_no_sw.iloc[index] = list(
        filter(
            lambda word: word not in stopwords.values,
            document
        )
    )

In [599]:
# Menampikan dokumen yang sudah ditokenisasi dan diproses
pprint(cleaned_documents_tokenized_no_sw[0])

['dokumen',
 'a',
 'sivitas',
 'akademika',
 'universitas',
 'indonesia',
 'menjalankan',
 'aktivitas',
 'perkuliahan',
 'daring',
 'diikuti',
 'mahasiswa',
 'jenjang',
 '.',
 'diakibatkan',
 'pandemi',
 'covid-19',
 'indonesia',
 '.']


In [600]:
pprint(cleaned_documents_tokenized_no_sw[1])

['masyarakat',
 'memiliki',
 'respon',
 'berbeda',
 'menyikapi',
 'pandemi',
 'covid-19',
 '.',
 'salah',
 'mahasiswa',
 'bernama',
 'dipsy',
 'kesulitan',
 'belajar',
 'maksimal',
 'sulit',
 'berdiskusi',
 'orang',
 'pandai',
 '.']


In [601]:
pprint(cleaned_documents_tokenized_no_sw[2])

['salah',
 'kunci',
 'menghadapi',
 'musibah',
 'bersabar',
 'musibah',
 '.',
 'badai',
 ',',
 'semangat',
 '.']


### 2. [10] Jika menjalankan masing-masing query berikut, dokumen manakah yang akan ditemukan?

In [602]:
DOCUMENTS_INDEX_DICT = {
    0: 'A',
    1: 'B',
    2: 'C',
}

### a. akibat AND sulit

In [603]:
doc_index = 0
is_result_found = False

for document in cleaned_documents_tokenized_no_sw:
    if (('akibat' in document) and ('sulit' in document)):
        print(f'Dokumen {DOCUMENTS_INDEX_DICT[doc_index]}')
        is_result_found = True
    doc_index += 1

if not is_result_found:
    print("Query tidak ditemukan dari dokumen manapun")

Query tidak ditemukan dari dokumen manapun


### b. waktu OR santai

In [604]:
doc_index = 0
is_result_found = False

for document in cleaned_documents_tokenized_no_sw:
    if (('waktu' in document) or ('santai' in document)):
        print(f'Dokumen {DOCUMENTS_INDEX_DICT[doc_index]}')
        is_result_found = True
    doc_index += 1

if not is_result_found:
    print("Query tidak ditemukan dari dokumen manapun")

Query tidak ditemukan dari dokumen manapun


### c. (masyarakat AND mahasiswa) OR sabar

In [605]:
doc_index = 0
is_result_found = False

for document in cleaned_documents_tokenized_no_sw:
    if ((('masyarakat' in document) and ('mahasiswa' in document)) or ('sabar') in document):
        print(f'Dokumen {DOCUMENTS_INDEX_DICT[doc_index]}')
        is_result_found = True
    doc_index += 1

if not is_result_found:
    print("Query tidak ditemukan dari dokumen manapun")

Dokumen B


### d. (libur OR respon) OR (waktu AND jenjang)

In [606]:
doc_index = 0
is_result_found = False

for document in cleaned_documents_tokenized_no_sw:
    if ((('libur' in document) or ('respon' in document)) or
        (('waktu') in document) and ('jenjang' in document)):
        print(f'Dokumen {DOCUMENTS_INDEX_DICT[doc_index]}')
        is_result_found = True
    doc_index += 1

if not is_result_found:
    print("Query tidak ditemukan dari dokumen manapun")

Dokumen B


### e. (kunci OR musibah) AND NOT (aktivitas AND pandemi) OR maksimal

In [607]:
doc_index = 0
is_result_found = False

for document in cleaned_documents_tokenized_no_sw:
    if ((('kunci' in document) or ('musibah' in document)) and not
        (('aktivitas') in document) and ('pandemi' in document) or
        ('maksimal') in document):
        print(f'Dokumen {DOCUMENTS_INDEX_DICT[doc_index]}')
        is_result_found = True
    doc_index += 1

if not is_result_found:
    print("Query tidak ditemukan dari dokumen manapun")

Dokumen B


### 3. [10] Apakah metode pemetaan dokumen dan teks dapat memberikan manfaat dalam proses perancangan sistem IR apabila dilihat dari perspektif keakuratan hasil yang akan dihasilkan? Jelaskan analisis singkat kamu minimal dalam 3 kalimat.